# Tyk 100 iterations, small dataset, worker=1

## Import packages

In [14]:
import pandas as pd
import altair as alt
import zipfile
import os
import shutil

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

## Copy html report from playwright to folder playwright-html

In [15]:
def copy_file(src_file, dest_folder):
    try:
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
        shutil.copy(src_file, dest_folder)
    except Exception as e:
        print(f"Error occurred: {e}")

src_file = '/Users/sonchau/Documents/CANDIG/playwright-runbook/playwright-report/index.html'
dest_folder = os.path.join(os.getcwd(), "playwright-html")
copy_file(src_file, dest_folder)

## Unzip report.zip from playwright to playwright-data

In [16]:
def unzip_file(zip_file):
    extract_to = os.path.join(os.getcwd(), "playwright-data")
    os.makedirs(extract_to, exist_ok=True)

    with zipfile.ZipFile(zip_file, "r") as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Extracted all files to: {extract_to}")


# Specify the path to the zip file
zip_file_path = (
    "/Users/sonchau/Documents/CANDIG/playwright-runbook/blob-report/report.zip"
)

unzip_file(zip_file_path)

Extracted all files to: /Users/sonchau/Documents/CANDIG/playwright-notebook/playwright-data


## Load data from file into dataframe

In [17]:
df_original = pd.read_json("./playwright-data/report.jsonl", lines=True)
# Skip the first 4 rows of metadata
df_without_metadata = df_original.iloc[4:].reset_index(drop=True)
# df_without_metadata.head()

# Get metadata

In [18]:
df_onEnd = df_original.loc[df_original["method"] == "onEnd"]
df_metadata = df_original.loc[df_original["method"] == "onProject"]
# df_metadata.head()

# Get title with testID

In [19]:
df_metadata_perf = pd.json_normalize(
    df_metadata.iloc[1]["params"],
    record_path=["project", "suites", "entries"],
)
df_title_with_id = df_metadata_perf[["testId", "title"]]
# df_title_with_id.head()

## Link duration for each test

In [20]:
df_params = pd.json_normalize(df_without_metadata["params"])
# Define the value to match
# filter_value = "apiRequestContext.get(http://localhost:8008/v3/" # katsu
# filter_value = "apiRequestContext.get(http://localhost:1236/" # query
# filter_value = "apiRequestContext.post(http://localhost:4232/" # federation
# filter_value = "http://localhost:3333/" # htsget
filter_value = "http://localhost:5080/" # htsget

# Filter rows where 'step.title' matches the value
df_filtered_value = df_params[
    df_params["step.title"].str.contains(filter_value, na=False, regex=False)
]
df_start_time_with_id = df_filtered_value[["step.id", "step.startTime", "testId"]]
df_duration = df_params[df_params["step.duration"].notnull()]

# Merge
df_start_time_with_id_and_duration = df_start_time_with_id.merge(
    df_duration[["step.id", "step.duration"]], on="step.id", how="left"
)
df_duration_with_id = df_start_time_with_id_and_duration[
    ["step.id", "step.duration", "testId"]
]
# df_duration_with_id.head()

## Link testId with title for a readable dataset

In [21]:
df_duration_with_id = df_duration_with_id.merge(
    df_title_with_id, on="testId", how="left", suffixes=("", "_y")
)

df_duration_with_title = df_duration_with_id[["title", "step.duration"]]
df_duration_with_title = df_duration_with_title.rename(
    columns={"step.duration": "duration"}
)
# df_duration_with_title.head()

## Basic analysis

In [22]:
# Total time
df_normalized = pd.json_normalize(df_onEnd['params'])
total_time = df_normalized['result.duration'] / 1000
print(f"Total time: {total_time.iloc[0]:.2f} s")

# Define the custom percentiles function
def get_p90(x):
    return x.quantile(0.90)
def get_p95(x):
    return x.quantile(0.95)
# Group by 'title' and calculate the desired statistics
result = (
    df_duration_with_title.groupby("title")["duration"]
    .agg(min="min", max="max", median="median", p90=get_p90, p95=get_p95, count="count")
    .reset_index()
)
result.sort_values(by="median", ascending=True)



Total time: 195.06 s


,title,min,max,median,p90,p95,count
0,beacon gene_id,11.0,9979.0,202.5,1200.3,2482.15,100
5,service-info,19.0,15195.0,223.5,1169.3,2066.90,100
1,beacon pos,15.0,2909.0,229.5,711.8,2029.85,100
4,samples,58.0,14910.0,701.0,1931.7,4524.65,100
6,variant,107.0,11163.0,738.0,2064.9,3607.20,100
3,genes,381.0,20072.0,1904.0,10330.7,13950.80,100
2,completeness,806.0,22745.0,5499.0,8845.3,17531.95,100


## Boxplot

In [23]:
box_plot = (
    alt.Chart(df_duration_with_title, title="API timings")
    .mark_boxplot(size=30)
    .encode(
        y=alt.Y("title:N").title("Endpoints"),
        x=alt.X("duration:Q").title("Duration (ms)"),
        color=alt.Color("title:N").legend(None),
        tooltip=["duration:Q"],
    )
    .properties(width=800, height=400)
)
box_plot.show()

alt.Chart(...)

# Histogram

In [24]:
histogram = (
    alt.Chart(df_duration_with_title)
    .mark_bar()
    .encode(
        alt.X("duration:Q", bin=True, title="Duration (ms)"),
        alt.Y("count():Q", title="Count"),
        alt.Color("title:N", legend=None),
        tooltip=[alt.Tooltip("count():Q", title="Count")],
    )
    .facet("title:N", columns=3, title="API timings")
    .resolve_axis(x='independent')
)

histogram.show()

alt.FacetChart(...)

# Timeline

In [25]:
df_with_timestamp = df_start_time_with_id_and_duration.merge(df_title_with_id, on='testId', how='left', suffixes=('', '_y'))
df_timeline = df_with_timestamp[['title','step.duration', 'step.startTime']]
df_timeline = df_timeline.rename(columns={'step.duration': 'duration'})
# df_duration_with_title_with_timestamp

# Line chart

In [26]:
df_timeline['relativeTime'] = (df_timeline['step.startTime'] - df_timeline['step.startTime'].min())
chart = alt.Chart(df_timeline).mark_line(point=True).encode(
    x=alt.X('relativeTime:Q', title='Period'),
    y=alt.Y('duration:Q', title='Response time (ms)'),
    color='title:N',
    tooltip=['title', 'duration', 'relativeTime']
).properties(
    title='Response time over period',
    width=800,
    height=500 
)

chart.display()

alt.Chart(...)